2. Explain how you would integrate it into the training pipeline __(text needed)__
    * When/ Where are the negative samples going to be generated?
    * How are they going to be used and what loss functions will you use and why?
    * How could it affect your evaluation pipeline?

## When/ Where are the negative samples going to be generated?

When?  

Negative samples, also known as synthetic negatives or false facts, are generated mostly during the training phase of the model. The negative examples are created by corrupting either the subject or the object of a positive triple while keeping the predicate unchanged. This is done to introduce **false facts that the model must learn to distinguish from true facts**, as most of the knowledge graphs like wikipedia dont have the false facts inherent wihtin them.

**Local Closed World Assumption (LCWA):** The assumption is that each triple is locally complete, and any corruption (e.g., replacing a tail entity with a random entity) is treated as a **negative sample**.  
This avoids the open-world assumption issue, where a missing fact is not inherently false but unknown.

<img src="./resources/NegativeSampling.png" alt="NegativeSampling" width="500"/>

Where?  


**Negative sampling is done on-the-fly during training:** For each positive triple (head, relation, tail), we generate negative samples by corrupting either the subject or the object but never the relation. This ensures that the negative samples remain realistic and are simply corrupted versions of the original true facts.

    Common methodologies:
        - Batch-wise sampling
        - Models using contrastive loss incorporates within loss calculation step
        - Pre-processing step(less common) - a static set creation 

Examples of Generation:

- Corrupt the object: Replace "Liverpool" in the triple (Mike, born_in, Liverpool) with another entity, e.g., (Mike, born_in, Acme Inc.), to create a false fact.
- Corrupt the subject: Replace "Mike" in the triple with another entity, e.g., (Acme Inc., born_in, Liverpool).

Purpose: These negatives are used to train the model to differentiate between true and false facts and are generated repeatedly during the training loop to improve model performance.


## How are they going to be used and what loss functions will you use and why?

Self-adversarial negative sampling loss is the best choice for a ComplEx model with data-informed negative sampling because:

![Sun.et.al](./resources/sunetal-loss.png)  


- **Focuses on hard negatives:** It dynamically **assigns higher weights to negative samples that the model finds difficult**, allowing the model to focus on challenging examples.
- **Better suited for data-informed sampling:** Since negative samples are generated based on **semantic or structural relevance**, this loss function ensures that the model concentrates on the most informative negatives, rather than treating all negatives equally.
- **More scalable:** While hinge loss and cross-entropy require processing every negative sample uniformly, self-adversarial loss focuses on a subset of hard negatives, reducing computational overhead and improving efficiency in large-scale knowledge graphs.
- **Tailored for ComplEx:** ComplEx models capture complex interactions between entities and relations, and self-adversarial loss enhances learning by emphasizing subtle distinctions between positive and hard negative triples.
- **Contrast with hinge loss:** Hinge loss treats all **negatives equally with a fixed margin**, which is less effective in scenarios where negatives are generated based on meaningful data insights.
- **Contrast with binary cross-entropy:** Binary cross-entropy **lacks a ranking mechanism** and does not dynamically prioritize negatives, making it less suitable for models like ComplEx that require fine-grained ranking between positive and negative triples.

Self-adversarial loss optimally leverages data-informed negative sampling to improve model performance and generalization.

## How could it affect your evaluation pipeline?

Negative sampling can significantly impact the evaluation pipeline, particularly in **ranking-based metrics like Mean Rank (MR), Mean Reciprocal Rank (MRR), and Hits@k**. During evaluation, synthetic negatives are generated by corrupting either the subject or object entities, just as during training. The model’s performance is then assessed by how well it ranks the true (positive) triples against these negatives.

**Below image shows, how the metrics value gets impacted because of the negative samples:**  
<img src="./resources/Example_Metrics.png" alt="Example" width="300"/>

If the negatives are too easy, evaluation metrics **might overestimate the model's performance**, as distinguishing true triples from trivial false ones doesn’t challenge the model. Conversely, focusing on hard negatives (those that are difficult to differentiate from true facts) can lower evaluation scores, even if the model generalizes well. Thus, the choice of negative sampling strategy directly influences how reliable and realistic the evaluation is.

Negative sampling also affects the **scalability of evaluation**. Generating a large number of negatives for each test triple can be computationally expensive. Therefore, batching or limiting the number of negatives becomes necessary to avoid overloading the system.

**How Could Calibration Work with Negative Samples?**  

Calibration ensures that the model’s output scores accurately reflect probabilities. In knowledge graphs, synthetic negatives are used for this, as explicit negative facts are usually unavailable. 

<img src="./resources/calibration.png" alt="calibration" width="500"/>  

Techniques like Platt scaling or isotonic regression can be applied to adjust model scores. Combining positives and synthetic negatives helps the model assign lower probabilities to false facts, making the output more interpretable and trustworthy. Calibrated models improve decision-making, ensuring that the predicted probabilities align with real-world outcomes.